<a href="https://colab.research.google.com/github/GrueneKatze/AI-for-Medicine-Coursera/blob/master/Copy_of_Colab_Tips_%26_Useful_Snippets_06_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##FastAI updates, imports, mounting Google Drive

In [ ]:
!curl -s https://course.fast.ai/setup/colab | bash

In [ ]:
from fastai.vision import *
from fastai.metrics import error_rate
import os

In [ ]:
!pip install -q pydicom

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from matplotlib.patches import Rectangle
import seaborn as sns
import pydicom as pydicom

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'fastai/'

In [ ]:
#set path
path_r = Path("/content/data")

In [ ]:
path_rsna = Path(base_dir + 'rsna')
path_rsna.mkdir(parents=True, exist_ok=True)

In [ ]:
#Check content in a folder
path_r.ls()

In [ ]:
#Moving files from one folder to another
files = os.listdir(path_train)
source = '/content/gdrive/My Drive/fastai/rsna/train/'
dest1 = '/content/gdrive/My Drive/fastai/rsna/valida/'
for f in files:
  shutil.move(source+f, dest1)

##Zip, to/ from csv, dcm

In [ ]:
import zipfile

In [ ]:
with zipfile.ZipFile(path_f/'stage_2_detailed_class_info.csv.zip', 'r') as zip_ref:
  zip_ref.extractall(path_rsna)

In [ ]:
train_labels_df.to_csv(path_train/'train_labels.csv')

In [ ]:
train_labels_df = pd.read_csv(path_rsna/'stage_2_train_labels.csv')

In [ ]:
def open_dcm_image(fn:PathOrStr,convert_mode:str='RGB',after_open:Callable=None)->Image:
  "Return `Image` object created from image in file `fn`."
  array = pydicom.dcmread(fn).pixel_array
  x = PIL.Image.fromarray(array).convert('RGB')
  return Image(pil2tensor(x,np.float32).div_(255))

In [ ]:
vision.data.open_image = open_dcm_image

##Dataframes

In [ ]:
#Creating empty dataframe with same columns
valid_labels_df = pd.DataFrame(columns=train_labels_df.columns)

In [ ]:
#Check data types in df
df.dtypes

In [ ]:
#replacing values in a column based on condition
data_w_dummies.loc[data_w_dummies['Churn'] == 'Yes', 'Churn'] = 1

In [ ]:
#excluding raws based on condition
df_filtered = df[df['Age'] >= 25]

In [ ]:
#Moving rows based on condition
for i in image_ids: #list of patientId values based on which we move rows
  #if i == train_labels_df.patientId:
  rows = train_labels_df.loc[train_labels_df['patientId'] == i, :]
  valid_labels_df = valid_labels_df.append(rows, ignore_index=False)
  train_labels_df.drop(rows.index, inplace=True)


In [ ]:
#Showing a sample 
train_labels_df.sample(10)

In [ ]:
#Checking if there're nans
class_info_df.isnull().any()

In [ ]:
#Checking if there're any empty cells
#np.where(data_w_dummies.applymap(lambda x: x == ''))
df[df['column_name'] == ''].index

In [ ]:
#creating list of nan indexes
def check_nulls(data):
  check = data.isnull().any()
  res = []
  for i in check:
    if i== True:
      res.append(check[check == i].index[0])
  return res

#deleting nans; cols = output of check_nulls(data)
def delete_nulls(cols, data):
  for i in cols:
    checked_data = data
    #data = checked_data
    null_to_find = pd.isnull(checked_data[i])
    raw = checked_data[null_to_find].index[0]
    data = checked_data.drop(raw)
  return data


In [ ]:
#Merging 2 datasets (inner)

train_class_df = train_labels_df.merge(class_info_df, left_on='patientId', right_on='patientId', how='inner')

In [ ]:
#convert class data into one-hot encoding format
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
class_enc = LabelEncoder()
train_class_df['class_idx'] = class_enc.fit_transform(train_class_df['class'])
oh_enc = OneHotEncoder(sparse=False)
train_class_df['class_vec'] = oh_enc.fit_transform(train_class_df['class_idx'].values.reshape(-1, 1)).tolist() 
train_class_df.sample(3)

In [ ]:
#convert categorical values into multiple columns
dummy_fields = ['gender', 'Partner', 'Dependents','PhoneService','MultipleLines']
for each in dummy_fields:
    dummies = pd.get_dummies(dataset[each], prefix=each, drop_first=False)
    dataset = pd.concat([dummies, dataset], axis=1)

data_w_dummies = dataset.drop(dummy_fields, axis=1)
data_w_dummies.sample()

In [ ]:
#convert column values to float
weather["Temp"] = weather.Temp.astype(float)

#or

data_w_dummies['TotalCharges'] = pd.to_numeric(data_w_dummies['TotalCharges'])

## Visualizations

In [ ]:
#corr matrix
corrmat = data.corr()

cols = ['instant','dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed']
k = 15 #number of variables for heatmap
cols = corrmat.nlargest(k, 'cnt')['cnt'].index
cm = np.corrcoef(data[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

##Getting file names

In [ ]:
fnames = get_files(path_train,extensions=['.dcm'],recurse=recurse)#(path_images)get_files

##Printing formats

In [ ]:
print(f"Train labels -  rows: {train_labels_df.shape[0]}, columns: {train_labels_df.shape[1]}")
print(f"your text you want to explain the output: {value_you_want_to_show}")

##Saving and loading models, finding proper learning rate

In [ ]:
learn.save(path_m/'name')

In [ ]:
learn.load(path_m/'name')

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

## Speeding up Colab

In [ ]:
#https://medium.com/@oribarel/getting-the-most-out-of-your-google-colab-2b0585f82403

#Checking GPU usage

!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 



  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=d3168b6cb8ec7f780c1fcc700e0c22641c2b158bb4b6c2911e8dde6dc6988603
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.7 GB  | Proc size: 159.5 MB
GPU RAM Free: 15079MB | Used: 0MB | Util   0% | Total 15079MB


In [ ]:
#Create placegholders for datasets
import h5py
from PIL import Image

fileName = 'data.h5'
numOfSamples = 10000
with h5py.File(fileName, "w") as out:
  
  out.create_dataset("X_train",(numOfSamples,256,256,3),dtype='u1')
  out.create_dataset("Y_train",(numOfSamples,1,1),dtype='u1')      
  out.create_dataset("X_dev",(numOfSamples,256,256,3),dtype='u1')
  out.create_dataset("Y_dev",(numOfSamples,1,1),dtype='u1')      
  out.create_dataset("X_test",(numOfSamples,256,256,3),dtype='u1')
  out.create_dataset("Y_test",(numOfSamples,1,1),dtype='u1')   

In [ ]:
#Load datasets to placeholders
with h5py.File(fileName, "a") as out:
   img = Image.open("X_train_1.jpg")      # X_train_1.jpg is 256 x 256 RGB image
   out['X_train'] = numpy.asarray(img)